<img src="https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/logo_sizinglab.png?raw=1" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Sizing of a multi-rotor drone

*Written by Marc Budinger, Aitor Ochotorena (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

The objective of this notebook is to select the best compromise of components (propeller, motor, ESC, battery) of a multi-rotor drone for given specifiations.

**Scipy** and **math** packages will be used for this notebook in order to illustrate the optimization algorithms of python.

In [196]:
import scipy
import scipy.optimize
from math import pi
from math import sqrt
from math import sin
from math import *
import math
import numpy as np
import timeit
import pandas as pd
from IPython.display import display, HTML

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

pd.options.display.float_format = '{:,.2f}'.format

## Sizing code

The set of equations of a sizing code can generate typical issues such : 
- Underconstrained set of equations: the lacking equations can come from additional scenarios, estimation models or additional sizing variable.   
- overconstrained equations often due to the selection of a component on multiple critera: the adding of over-sizing coefficients and constraints in the optimization problem can generally fix this issue   
- algebraic loops often due to selection criteria requiring informations generally available after the selection 

Concerning overconstraints components, we have here:
- Brushless motors with multiple torque and voltage constraints (hover and transient vertical displacement) 

Multiple algebraic loops appears in the sizing problem:
- The thrust depends of the total mass which depend of components required for generating this thrust

The final optimization problem depends thus of these parameters:
- $\beta_{pro}=pitch/diameter$ ratio to define the propeller
- $k_{os}$ over sizing coefficient on the load mass to estimate the final total mass
- $k_{mot}$ over sizing coeffcient on the motor torque to estimate the max torque with the hover flight conditions
- $k_{speed,mot}$ over sizing coeffcient on the motor speed to take into account voltage limits during hover or take-off flight
- $k_{ND}$ slow down propeller coef : ND = kNDmax / k_ND
- $k_{D}$ aspect ratio e_arm/D_out_arm (thickness/diameter) for the beam of the frame
- $k_{mb}$ ratio battery mass / payload mass
- $k_{vb}$ over sizing coefficient for the battery voltage

More details in the setting up of sizing code can be found in the  [following paper](https://www.researchgate.net/profile/Marc_Budinger/publication/277933677_Computer-aided_definition_of_sizing_procedures_and_optimization_problems_of_mechatronic_systems/links/55969de508ae793d137c7ea5/Computer-aided-definition-of-sizing-procedures-and-optimization-problems-of-mechatronic-systems.pdf):  

> Reysset, A., Budinger, M., & Maré, J. C. (2015). Computer-aided definition of sizing procedures and optimization problems of mechatronic systems. Concurrent Engineering, 23(4), 320-332.

The sizing code is defined here in a function which can give:
- an evaluation of the objective: here the total mass
- an evaluation of the constraints: 

Here is an non-exhaustive XDSM diagram of the multirotor sizing code:

![XDSM](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/xdsm_multirotor_mdo.png?raw=1)

## Objectives and specifications

Main specifications :
- a load (video, control card) of mass $M_{load}$.  
- an autonomy $t_{hf}$ for the hover flight.
- an acceleration to take off $a_{to}$.


In [197]:
# Specifications

# Load
M_pay = 100. # [kg] load mass

# Acceleration during take off
a_to = 0.25 * 9.81 # [m/s²] acceleration

# Autonomy
t_hov_spec = 25. # [min] time of hover flight

# MTOW
MTOW = 360. # [kg] maximal mass

# Objectif
MAX_TIME = False # Objective



## Architecture definition and design assumptions

In [198]:
# Architecture of the multi-rotor drone (4,6, 8 arms, ...)
N_arm = 4 # [-] number of arm
N_pro_arm = 2 # [-] number of propeller per arm (1 or 2)
N_pro = N_pro_arm * N_arm # [-] Propellers number

## Reference parameters

- ### Battery & ESC

In [199]:
# Reference parameters for scaling laws
# Ref : MK-quadro
M_bat_ref = .329 # [kg] mass
E_bat_ref = 220.*3600.*.329 # [J]

# Ref : Turnigy K_Force 70HV 
P_esc_ref = 3108. # [W] Power
M_esc_ref = .115 # [kg] Mass

- ### Motor

In [200]:
# Motor reference
# Ref : AXI 5325/16 GOLD LINE
T_nom_mot_ref = 2.32  # [N.m] rated torque
T_max_mot_ref = 85./70.*T_nom_mot_ref # [N.m] max torque
R_mot_ref = 0.03  # [Ohm] resistance
M_mot_ref = 0.575 # [kg] mass
K_mot_ref = 0.03 # [N.m/A] torque coefficient
T_mot_fr_ref = 0.03 # [N.m] friction torque (zero load, nominal speed)

- ### Frame

In [201]:
# Reference parameters for scaling laws
sigma_max = 280e6/4. # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
rho_s = 1700. # [kg/m3] Volumic mass of aluminum

- ### Propeller

In [202]:
# Specifications
rho_air=1.18# [kg/m^3] Air density
ND_max=105000./60.*.0254 #[Hz.m] Max speed limit (N.D max) for APC MR propellers


# Reference parameters for scaling laws
D_pro_ref=11.*.0254# [m] Reference propeller diameter
M_pro_ref=0.53*0.0283# [kg] Reference propeller mass

In [203]:
# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else
def sizing_code(param, arg):
# Design variables
# ---
    beta_pro = param[0] # pitch/diameter ratio of the propeller
    k_os = param[1] # over sizing coefficient on the load mass 
    k_ND = param[2] # slow down propeller coef : ND = kNDmax / k_ND
    k_mot = param[3] # over sizing coefficient on the motor torque
    k_speed_mot = param[4] # over sizing coefficient on the motor speed
    k_mb = param[5] # ratio battery / payload mass 
    k_vb = param[6] # over sizing coefficient for the battery voltage
    k_D = param[7] # aspect ratio e_arm/D_out_arm (thickness/diameter) for the beam of the frame

# Hover& Take-Off thrust 
# ---
    M_total = k_os*M_pay # [kg] Estimation of the total mass (or equivalent weight of dynamic scenario)
    F_pro_hov = M_total*9.81/N_pro # [N] Thrust per propeller for hover
    F_pro_to = M_total*(a_to+9.81)/N_pro # [N] Thrust per propeller for take-off


#PROPELLER
# --- 
#Equations:
#-----
    ND = ND_max/k_ND

# Regression
    C_t = 4.27e-02 + 1.44e-01 * beta_pro # # Thrust coef with T=C_T.rho.n^2.D^4 - 0.8 for de-rating of APC catalog - Linear Regression from previous notebook 04_PropellerLinearRegression-Student.ipynb
    C_p = -1.48e-03 + 9.72e-02 * beta_pro # Power coef with P=C_p.rho.n^3.D^5 - Linear Regression from previous notebook 04_PropellerLinearRegression-Student.ipynb

# Propeller selection with take-off scenario

    D_pro = sqrt(F_pro_to/(rho_air * (ND**2) * C_t)) # [m] Propeller diameter - From equation Ct = F/rho*N^2*D^3
    n_pro_to = ND/D_pro # [Hz] Propeller speed
    Omega_pro_to = n_pro_to*2*pi # [rad/s] Propeller speed
    P_pro_to = C_p * rho_air * (ND**3) * D_pro**2 # From equation Cp = P/rho*N^3*D^5 and T = P/W
    T_pro_to = P_pro_to/Omega_pro_to 
    M_pro = M_pro_ref * (D_pro/D_pro_ref)**3 # From equation M* = D*^2 et var* = var/var_ref
    U_bat_est = k_vb*1.84*(P_pro_to)**(0.36) # [V] battery voltage estimation (from regression)

# Propeller torque & speed for hover
    D_pro_hov = sqrt(F_pro_hov/(rho_air * (ND**2) * C_t))  # From equation Ct = F/rho*N^2*D^3 
    n_pro_hov = ND/D_pro_hov
    Omega_pro_hov = n_pro_hov*2*pi
    P_pro_hov = C_p * rho_air * (ND**3) * D_pro_hov**2 # From equation Cp = P/rho*N^3*D^5 and T = P/W
    T_pro_hov = P_pro_hov/Omega_pro_hov
    M_pro_hov = M_pro_ref * (D_pro_hov/D_pro_ref)**3 # From equation M* = D*^2 et var* = var/var_ref

# MOTOR
# --- 

#Equations:
#-----
    T_nom_mot = k_mot * T_pro_hov  # [N.m] Motor nominal torque per propeller
    M_mot = M_mot_ref * (T_nom_mot/T_nom_mot_ref)**(3/3.5)  # [kg] Motor mass (scaling law)

# Selection with take-off speed
    K_mot = U_bat_est/Omega_pro_to/k_speed_mot # [N.m/A] or [V/(rad/s)] Kt motor
    R_mot = R_mot_ref * (K_mot/K_mot_ref)**2 * (T_nom_mot/T_nom_mot_ref)**(-5/3.5)  # [Ohm] motor resistance (scaling law)

    T_mot_fr = T_mot_fr_ref * (T_nom_mot/T_nom_mot_ref)**(3/3.5) # [N.m] Friction torque (scaling law)

    T_max_mot = T_max_mot_ref*(T_nom_mot/T_nom_mot_ref)

# Hover current and voltage
    I_mot_hov = (T_nom_mot+T_mot_fr) / K_mot # [I] Current of the motor per propeller
    U_mot_hov = R_mot * I_mot_hov + K_mot * Omega_pro_hov # [V] Voltage of the motor per propeller
    P_el_mot_hov = I_mot_hov * U_mot_hov  # [W] Hover : electrical power

# Takeoff current and voltage
    I_mot_to = (T_max_mot+T_mot_fr) / K_mot # [I] Current of the motor per propeller
    U_mot_to = R_mot * I_mot_to + K_mot * Omega_pro_to # [V] Voltage of the motor per propeller
    P_el_mot_to = I_mot_to * U_mot_to # [W] Takeoff : electrical power

# BATTERY AND ESC
# ---   
# Battery selection & scaling laws sized from hover
    
    
    N_s_bat = ceil(U_bat_est/3.7)  # [-] Cell number, round (up value).  from math import *
    U_bat = N_s_bat *3.7 # [V] Battery voltage

    M_bat = k_mb * M_pay # [kg] Battery mass


# Hover --> autonomy
    E_bat = E_bat_ref * M_bat / M_bat_ref * 0.8 # [J] Energy  of the battery (.8 coefficient because 80% use only of the total capacity)
    C_bat = E_bat/U_bat  # [A.s] Capacity  of the battery 
    I_bat =  P_el_mot_hov * N_pro / (0.95 * U_bat)  # [A] Current of the battery (Initial hypothese)
    t_hov = C_bat /(I_bat*60)  # [min] Hover time 


 #ESC    
  
    P_esc = P_el_mot_to * U_bat/U_mot_to # [W] power electronic power (corner power or apparent power)
    M_esc =  M_esc_ref * P_esc / P_esc_ref  # [kg] Mass ESC
    V_esc =  1.84e+00*P_esc**(3.60e-01) # [V] ESC voltage     

# Frame sized from max thrust
# ---
    alpha_sep = 2*pi/N_arm # [rad] interior angle separation between propellers
    L_arm =  D_pro/(2*math.sin(pi/N_arm)) # [m] length of the arm

    # Tube diameter & thickness
    D_out_arm = math.pow(F_pro_to*L_arm/(math.pi/32*(1-(1-k_D)**4)*sigma_max) , 1/3) # [m] outer diameter of the arm (hollow cylinder)
    e_arm = D_out_arm*k_D # [m] thickness of the arm (hollow cylinder)  
    D_in_arm = D_out_arm - e_arm # [m] inner diameter of the arm (hollow cylinder)

    # Mass
    M_arm = math.pi*(D_out_arm**2-D_in_arm**2)/4*L_arm*rho_s # [kg] mass of the arm (x1) (beams only)  composite
    M_frame = (N_arm*M_arm)/0.4 # [kg] mass of the frame (40% of total mass is the arms)
    M_body = M_frame*0.6
    M_lg = 0 # [kg] Initial hypothese

# Objective and Constraints sum up
# ---
    M_total_real= (M_pro + M_mot + M_esc)*N_pro + M_bat + M_frame + M_pay +M_lg

    if MAX_TIME == True:
            # One of the objectifs which is maximized the time of hover flight
            constraints = [
              M_total - M_total_real, # Mass limitation M_total_real compared to M_total (M_total>=M_total_real)
              U_bat - U_mot_to,
              T_max_mot - T_pro_to,# - T_mot_fr,# Magnetic limitation of the T_pro_to + T_max_fr compared to T_max_motor (T_max_motor>=T_pro_to + T_max_fr)
              U_bat - V_esc, # Limitation of the V_esc compared to U_bat (U_bat>=V_esc)
              V_esc - U_mot_to, # Limitation of the U_mot_to compared to V_esc (V_esc>=U_mot_to)
              #Hypotheses sigma=sigma_max # Limitation of the sigma compared to sigma_max (sigma_max>=sigma)
              MTOW - M_total_real 
              ]
    else:
            # One of the objectifs which is minimized the MTOW
            constraints = [
              M_total - M_total_real, # Mass limitation M_total_real compared to M_total (M_total>=M_total_real)
              U_bat - U_mot_to,
              T_max_mot - T_pro_to,# - T_mot_fr,# Magnetic limitation of the T_pro_to + T_max_fr compared to T_max_motor (T_max_motor>=T_pro_to + T_max_fr)
              U_bat - V_esc, # Limitation of the V_esc compared to U_bat (U_bat>=V_esc)
              V_esc - U_mot_to, # Limitation of the U_mot_to compared to V_esc (V_esc>=U_mot_to)
              #Hypotheses sigma=sigma_max # Limitation of the sigma compared to sigma_max (sigma_max>=sigma)
              t_hov - t_hov_spec
            ]

 # Run algorithm slsqp
    if arg == 'Obj':
        if MAX_TIME == True:
            return 1./t_hov # for time maximisation
        else:
            return M_total_real # for mass optimisation

     # Run algorithm differential evolution
    elif arg == 'ObjP':
        P = 0. # Penalisation nulle
        for C in constraints: 
            if (C < 0.): 
                P = P-1e9*C
        if MAX_TIME==True:
            return 1./t_hov + P # for time maximisation
        else:
            return M_total_real + P # for mass optimisation       

    elif arg=='Prt':

        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()
        print(constraints)
        df = df.append([{'Type': 'Optimization', 'Name': 'beta_pro', 'Value': beta_pro,  'Unit': '[-]', 'Comment': 'ratio pitch-to-diameter '}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_os',  'Value': k_os,  'Unit': '[-]', 'Comment': 'over sizing coefficient on the load mass '}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_ND',  'Value': k_ND,  'Unit': '[-]', 'Comment': 'over sizing coefficient on the propeller speed'}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_mot', 'Value': k_mot,  'Unit': '[-]', 'Comment': 'over sizing coefficient on the motor torque  '}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_speed_mot',  'Value': k_speed_mot,  'Unit': '[-]', 'Comment': 'over sizing coefficient on the propeller speed'}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_mb',  'Value': k_mb,  'Unit': '[-]', 'Comment': 'over sizing coefficient on the battery load mass'}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_vb',  'Value': k_vb,  'Unit': '[-]', 'Comment': 'over sizing coefficient for the battery voltage'}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_D',  'Value': k_D,  'Unit': '[-]', 'Comment': 'aspect ratio e/c (thickness/side) for the beam of the frame'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 0',  'Value': constraints[0],  'Unit': '[-]', 'Comment': 'M_total-M_total_real'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 1',  'Value': constraints[1],  'Unit': '[-]', 'Comment': 'U_bat-U_mot_to'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 2',  'Value': constraints[2],  'Unit': '[-]', 'Comment': 'T_max_mot-T_pro_to'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 3',  'Value': constraints[3],  'Unit': '[-]', 'Comment': 'U_bat-V_esc'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 4',  'Value': constraints[4],  'Unit': '[-]', 'Comment': 'V_esc-U_mot_to'}])[col_names]
        if MAX_TIME==False:
            df = df.append([{'Type': 'Constraints', 'Name': 'Const 5',  'Value': constraints[4],  'Unit': '[-]', 'Comment': 't_hov-t_hov_spec'}])[col_names]
        else:
            df = df.append([{'Type': 'Constraints', 'Name': 'Const 5',  'Value': constraints[4],  'Unit': '[-]', 'Comment': 'MTOW-M_total_real'}])[col_names]
        df = df.append([{'Type': 'Objective', 'Name': 'Objective',  'Value': M_total_real,  'Unit': '[kg]', 'Comment': 'Total mass'}])[col_names]
        
        df = df.append([{'Type': 'Propeller', 'Name': 'F_pro_to', 'Value': F_pro_to, 'Unit': '[N]', 'Comment': 'Thrust for 1 propeller during Take Off'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'F_pro_hov', 'Value': F_pro_hov, 'Unit': '[N]', 'Comment': 'Thrust for 1 propeller during Hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'rho_air', 'Value': rho_air, 'Unit': '[kg/m^3]', 'Comment': 'Air density'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'ND_max', 'Value': ND_max, 'Unit': '[Hz.m]', 'Comment': 'Max speed limit (N.D max)'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Dpro_ref', 'Value': D_pro_ref, 'Unit': '[m]', 'Comment': 'Reference propeller diameter'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'M_pro_ref', 'Value': M_pro_ref, 'Unit': '[kg]', 'Comment': 'Reference propeller mass'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_t_sta', 'Value': C_t, 'Unit': '[-]', 'Comment': 'Static thrust coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_p_sta', 'Value': C_p, 'Unit': '[-]', 'Comment': 'Static power coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'D_pro', 'Value': D_pro, 'Unit': '[m]', 'Comment': 'Diameter of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'n_pro_to', 'Value': n_pro_to, 'Unit': '[Hz]', 'Comment': 'Rev speed of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'n_pro_hov', 'Value': n_pro_hov, 'Unit': '[Hz]', 'Comment': 'Rev speed of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'P_pro_to', 'Value': P_pro_to, 'Unit': '[W]', 'Comment': 'Power on the mechanical shaft of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'P_pro_hov', 'Value': P_pro_hov, 'Unit': '[W]', 'Comment': 'Power on the mechanical shaft of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'M_pro', 'Value': M_pro, 'Unit': '[kg]', 'Comment': 'Mass of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Omega_pro_to', 'Value': Omega_pro_to, 'Unit': '[rad/s]', 'Comment': 'Rev speed of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Omega_pro_hov', 'Value': Omega_pro_hov, 'Unit': '[rad/s]', 'Comment': 'Rev speed of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'T_pro_hov', 'Value': T_pro_hov, 'Unit': '[N.m]', 'Comment': 'Torque on the mechanical shaft of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'T_pro_to', 'Value': T_pro_to, 'Unit': '[N.m]', 'Comment': 'Torque on the mechanical shaft of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_max_mot_ref', 'Value': T_max_mot_ref, 'Unit': '[N.m]', 'Comment': 'Max torque'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'R_mot_ref', 'Value': R_mot_ref, 'Unit': '[Ohm]', 'Comment': 'Resistance'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'M_mot_ref', 'Value': M_mot_ref, 'Unit': '[kg]', 'Comment': 'Reference motor mass'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'K_mot_ref', 'Value': K_mot_ref, 'Unit': '[N.m/A]', 'Comment': 'Torque coefficient'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_mot_fr_ref', 'Value': T_mot_fr_ref, 'Unit': '[N.m]', 'Comment': 'Friction torque (zero load, nominal speed)'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_nom_mot', 'Value': T_nom_mot, 'Unit': '[N.m]', 'Comment': 'Continuous of the selected motor torque'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_max_mot', 'Value': T_max_mot, 'Unit': '[N.m]', 'Comment': 'Transient torque possible for climbing'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'R_mot', 'Value': R_mot, 'Unit': '[Ohm]', 'Comment': 'Resistance'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'M_mot', 'Value': M_mot, 'Unit': '[kg]', 'Comment': 'Motor mass'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'K_mot', 'Value': K_mot, 'Unit': '[N.m/A', 'Comment': 'Torque constant of the selected motor'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_mot_fr', 'Value': T_mot_fr, 'Unit': '[N.m]', 'Comment': 'Friction torque of the selected motor'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'I_mot_hov', 'Value': I_mot_hov, 'Unit': '[A]', 'Comment': 'Motor current for hover'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'I_mot_to', 'Value': I_mot_to, 'Unit': '[A]', 'Comment': 'Motor current for takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'U_mot_to', 'Value': U_mot_to, 'Unit': '[V]', 'Comment': 'Motor voltage for takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'U_mot', 'Value': U_mot_hov, 'Unit': '[V]', 'Comment': 'Nominal voltage '}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'P_el_mot_to', 'Value': P_el_mot_to, 'Unit': '[W]', 'Comment': 'Motor electrical power for takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'P_el_mot_hov', 'Value': P_el_mot_hov, 'Unit': '[W]', 'Comment': 'Motor electrical power for hover'}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_bat_ref', 'Value': M_bat_ref, 'Unit': '[kg]', 'Comment': 'Mass of the reference battery '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_esc_ref', 'Value': M_esc_ref, 'Unit': '[kg]', 'Comment': 'Reference ESC mass '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'P_esc_ref', 'Value': P_esc_ref, 'Unit': '[W]', 'Comment': 'Reference ESC power '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'N_s_bat', 'Value': np.ceil(N_s_bat), 'Unit': '[-]', 'Comment': 'Number of battery cells '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'U_bat', 'Value': U_bat, 'Unit': '[V]', 'Comment': 'Battery voltage '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_bat', 'Value': M_bat, 'Unit': '[kg]', 'Comment': 'Battery mass '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'C_bat', 'Value': C_bat, 'Unit': '[A.s]', 'Comment': 'Battery capacity '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'I_bat', 'Value': I_bat, 'Unit': '[A]', 'Comment': 'Battery current '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 't_hf', 'Value': t_hov, 'Unit': '[min]', 'Comment': 'Hovering time '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'P_esc', 'Value': P_esc, 'Unit': '[W]', 'Comment': 'Power electronic power (corner power or apparent power) '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_esc', 'Value': M_esc, 'Unit': '[kg]', 'Comment': 'ESC mass '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'V_esc', 'Value': V_esc, 'Unit': '[V]', 'Comment': 'ESC voltage '}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'N_arm', 'Value': N_arm, 'Unit': '[-]', 'Comment': 'Number of arms '}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'N_pro_arm', 'Value': N_pro_arm, 'Unit': '[-]', 'Comment': 'Number of propellers per arm '}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'sigma_max', 'Value': sigma_max, 'Unit': '[Pa]', 'Comment': 'Max admisible stress'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'L_arm', 'Value': L_arm, 'Unit': '[m]', 'Comment': 'Length of the arm'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'D_out', 'Value': D_out_arm, 'Unit': '[m]', 'Comment': 'Outer diameter of the arm (tube)'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'Marm', 'Value': M_arm, 'Unit': '[kg]', 'Comment': '1 Arm mass'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'M_frame', 'Value': M_frame, 'Unit': '[kg]', 'Comment': 'Frame mass'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'M_load', 'Value': M_pay, 'Unit': '[kg]', 'Comment': 'Payload mass'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 't_hf', 'Value': t_hov, 'Unit': '[min]', 'Comment': 'Hovering time '}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'N_arm', 'Value': N_arm, 'Unit': '[-]', 'Comment': 'Number of arms '}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'N_pro_arm', 'Value': N_pro_arm, 'Unit': '[-]', 'Comment': 'Number of propellers per arm '}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'MTOW', 'Value': MTOW, 'Unit': '[kg]', 'Comment': 'Max takeoff Weight'}])[col_names]

        items = sorted(df['Type'].unique().tolist())
        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
        
    else:
        return constraints

## Optimization problem


We will now use the [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLSQP algorithm without explicit expression of the gradient (Jacobian). A course on Multidisplinary Gradient optimization algorithms and gradient optimization algorithm is given [here](http://mdolab.engin.umich.edu/sites/default/files/Martins-MDO-course-notes.pdf):
> Joaquim R. R. A. Martins (2012). A Short Course on Multidisciplinary Design Optimization. University of Michigan


The first step is to give an initial value of optimisation variables:

In [204]:
# Optimisation variables
beta_pro = .33 # pitch/diameter ratio of the propeller
k_os = 3.2 # over sizing coefficient on the load mass 
k_ND = 1.2 # slow down propeller coef : ND = kNDmax / k_ND
k_mot = 1. # over sizing coefficient on the motor torque
k_speed_mot = 1.2 # adaption winding coef on the motor speed
k_mb = 1. # ratio battery/load mass
k_vb = 1. # oversizing coefficient for voltage evaluation
k_D = .01 # aspect ratio e/c (thickness/side) for the beam of the frame


# Vector of parameters
parameters = scipy.array((beta_pro, k_os, k_ND,k_mot, k_speed_mot, k_mb, k_vb,k_D,))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  del sys.path[0]


We can print of the characterisitcs of the problem before optimization with the initial vector of optimization variables:

In [205]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")
sizing_code(parameters,'Prt')
print("-----------------------------------------------")

-----------------------------------------------
Initial characteristics before optimization :
[31.306776862220318, 0.638904619054351, -6.361397830568592, 0.411144932480326, 0.227759686574025, -3.4882449210950064]


interactive(children=(Dropdown(description='Type', options=('Battery & ESC', 'Constraints', 'Frame', 'Motor', …

-----------------------------------------------


Then we can solve the problem and print of the optimized solution:

In [206]:
# Optimization with SLSQP algorithm
contrainte = lambda x: sizing_code(x, 'Const')
objectif = lambda x: sizing_code(x, 'Obj')
objectifP = lambda x: sizing_code(x, 'ObjP')

SLSQP = False # Optimization algorithm choice

# Optimization bounds
# beta,  k_os, k_ND, k_mot, k_speed_mot, k_mb, k_vb, k_D
bounds = [(0.3,0.6), (1,400), (1,100), (1,100), (1,400), (0.1,100), (1,5), (0.1,0.99)]

if SLSQP == True:
    # SLSQP omptimisation
    result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)
else:
    # Differential evolution omptimisation
    result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=bounds,
                                   tol=1e-12)

# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

if SLSQP == True:
    sizing_code(result,'Obj')
    sizing_code(result, 'Prt')
else:
    sizing_code(result.x,'Obj')
    sizing_code(result.x, 'Prt')
print("-----------------------------------------------")

-----------------------------------------------
Final characteristics after optimization :
[0.0017256311553524029, 2.4503438068140895, 9.203373439703455e-05, 2.428909485639501, 0.021434321174588433, 0.00012928263621603264]


interactive(children=(Dropdown(description='Type', options=('Battery & ESC', 'Constraints', 'Frame', 'Motor', …

-----------------------------------------------
